In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "6, 7"

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "/data/cuiluyi/resources/models/Qwen/Qwen2.5-Math-7B-Instruct"
# model_name = "/data/cuiluyi/resources/models/Qwen/Qwen2.5-Math-1.5B"
device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/data/cuiluyi/anaconda3/envs/open_reasoner/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.04s/it]


In [ ]:
system_prompt = "please think step by step"
user_prompt = """
Please delete the redundant content in the following solution:

To determine the smallest possible value of \\( c \\) for the function \\( y = a \\sin (bx + c) + d \\), we need to analyze the given graph and extract information about the parameters \\( a \\), \\( b \\), \\( c \\), and \\( d \\).\n\nFirst, observe the amplitude \\( a \\). The amplitude is the maximum deviation from the midline of the sine wave. From the graph, the maximum value is 3 and the minimum value is -1. The midline is the average of these values:\n\\[\n\\text{Midline} = \\frac{3 + (-1)}{2} = 1\n\\]\nThe amplitude \\( a \\) is the distance from the midline to the maximum value:\n\\[\na = 3 - 1 = 2\n\\]\n\nNext, determine the period of the sine wave. The period \\( T \\) is the length of one complete cycle of the sine wave. From the graph, the period appears to be \\( \\frac{2\\pi}{3} \\). The period of the sine function \\( y = a \\sin (bx + c) + d \\) is given by:\n\\[\nT = \\frac{2\\pi}{b}\n\\]\nSetting this equal to the observed period:\n\\[\n\\frac{2\\pi}{b} = \\frac{2\\pi}{3}\n\\]\nSolving for \\( b \\):\n\\[\nb = 3\n\\]\n\nNow, determine the vertical shift \\( d \\). The vertical shift is the value of the midline:\n\\[\nd = 1\n\\]\n\nFinally, determine the phase shift \\( c \\). The phase shift is the horizontal shift of the sine wave. The standard sine function \\( y = \\sin x \\) starts at \\( x = 0 \\). For the function \\( y = 2 \\sin (3x + c) + 1 \\), the phase shift is given by:\n\\[\n\\text{Phase shift} = -\\frac{c}{b} = -\\frac{c}{3}\n\\]\nFrom the graph, the sine wave starts at \\( x = -\\frac{\\pi}{6} \\). Therefore, the phase shift is:\n\\[\n-\\frac{c}{3} = -\\frac{\\pi}{6}\n\\]\nSolving for \\( c \\):\n\\[\nc = \\frac{\\pi}{2}\n\\]\n\nThus, the smallest possible value of \\( c \\) is:\n\\[\n\\boxed{\\frac{\\pi}{2}}\n\\]
"""
# """
# Role: Math Solution Simplifier

# ## Profile
# - **Purpose**: Condense mathematical solutions by removing redundant steps while preserving logical flow and accuracy.  
# - **Skills**:  
#   - Identifies core reasoning steps in proofs/solutions.  
#   - Eliminates repetitive calculations or verbose explanations.  
#   - Maintains mathematical rigor and clarity.  
# - **Knowledge**: Formal logic, common mathematical redundancies (e.g., over-explained axioms), and STEM writing conventions.  

# ## Rules
# 1. **Input**: A detailed mathematical solution (LaTeX/markdown/text).  
# 2. **Output**: A streamlined version with:  
#    - Only essential steps.  
#    - No repeated formulas/definitions.  
#    - Concise phrasing (e.g., "∵ symmetry" vs. "due to the symmetric property of...").  
# 3. **Preserve**: Key conclusions, non-trivial deductions, and original notation.  

# ## Workflow
# 1. Parse the solution into logical segments.  
# 2. For each segment:  
#    - **Check**: Is this step *necessary* for correctness or clarity?  
#    - **Cut**: Explanations of basic concepts, intermediate arithmetic.  
#    - **Keep**: Novel insights, critical dependencies.  
# 3. Reconstruct the solution with minimal viable steps.  

# ## Output Format
# ```markdown
# ### Simplified Solution  
# [Condensed steps, 30-50% shorter than original]
# ```  

# ## Example (Input → Output)

# Note: Output the simplified solution directly.

# ### Input
# To find the result of rotating the complex number \\( z = 2 + \\sqrt{2} - (3 + 3 \\sqrt{2})i \\) around the complex number \\( c = 2 - 3i \\) by \\(\\frac{\\pi}{4}\\) counter-clockwise, we need to follow these steps:\n\n1. **Translate \\( z \\) so that \\( c \\) is at the origin:**\n   \\[\n   z' = z - c\n   \\]\n   Substituting the given values:\n   \\[\n   z' = (2 + \\sqrt{2} - (3 + 3 \\sqrt{2})i) - (2 - 3i)\n   \\]\n   Simplify the real and imaginary parts separately:\n   \\[\n   z' = (2 + \\sqrt{2} - 2) + (- (3 + 3 \\sqrt{2})i + 3i)\n   \\]\n   \\[\n   z' = \\sqrt{2} - (3 + 3 \\sqrt{2} - 3)i\n   \\]\n   \\[\n   z' = \\sqrt{2} - 3 \\sqrt{2} i\n   \\]\n\n2. **Rotate \\( z' \\) by \\(\\frac{\\pi}{4}\\) counter-clockwise:**\n   To rotate a complex number \\( z' = a + bi \\) by an angle \\(\\theta\\), we multiply it by \\( e^{i\\theta} \\). For \\(\\theta = \\frac{\\pi}{4}\\):\n   \\[\n   e^{i\\frac{\\pi}{4}} = \\cos\\left(\\frac{\\pi}{4}\\right) + i \\sin\\left(\\frac{\\pi}{4}\\right) = \\frac{\\sqrt{2}}{2} + i \\frac{\\sqrt{2}}{2}\n   \\]\n   Therefore, the rotation is:\n   \\[\n   z'' = z' \\cdot e^{i\\frac{\\pi}{4}}\n   \\]\n   Substituting \\( z' = \\sqrt{2} - 3 \\sqrt{2} i \\):\n   \\[\n   z'' = (\\sqrt{2} - 3 \\sqrt{2} i) \\left( \\frac{\\sqrt{2}}{2} + i \\frac{\\sqrt{2}}{2} \\right)\n   \\]\n   Use the distributive property (FOIL method):\n   \\[\n   z'' = \\sqrt{2} \\cdot \\frac{\\sqrt{2}}{2} + \\sqrt{2} \\cdot i \\frac{\\sqrt{2}}{2} - 3 \\sqrt{2} i \\cdot \\frac{\\sqrt{2}}{2} - 3 \\sqrt{2} i \\cdot i \\frac{\\sqrt{2}}{2}\n   \\]\n   Simplify each term:\n   \\[\n   z'' = \\frac{2}{2} + i \\frac{2}{2} - i \\frac{6}{2} - 3 \\cdot \\frac{2}{2} i^2\n   \\]\n   \\[\n   z'' = 1 + i - 3i + 3\n   \\]\n   Combine like terms:\n   \\[\n   z'' = 4 - 2i\n   \\]\n\n3. **Translate back to the original coordinate system:**\n   \\[\n   w = z'' + c\n   \\]\n   Substituting the values:\n   \\[\n   w = (4 - 2i) + (2 - 3i)\n   \\]\n   Simplify the real and imaginary parts separately:\n   \\[\n   w = 4 + 2 - 2i - 3i\n   \\]\n   \\[\n   w = 6 - 5i\n   \\]\n\nThus, the result of rotating \\( z \\) around \\( c \\) by \\(\\frac{\\pi}{4}\\) counter-clockwise is:\n\\[\n\\boxed{6 - 5i}\n\\]

# ### Output
# To rotate the complex number  \n\\[\nz = 2 + \\sqrt{2} - (3 + 3\\sqrt{2})i\n\\]  \naround  \n\\[\nc = 2 - 3i\n\\]  \nby \\( \x0crac{\\pi}{4} \\) counter-clockwise:\n\n1. **Translate \\( z \\) so that \\( c \\) is at the origin:**  \n\\[\nz' = z - c = (2 + \\sqrt{2} - (3 + 3\\sqrt{2})i) - (2 - 3i) = \\sqrt{2} - 3\\sqrt{2}i\n\\]\n\n2. **Rotate \\( z' \\) by \\( \x0crac{\\pi}{4} \\):**  \nMultiplying by \\( e^{i\\pi/4} = \x0crac{\\sqrt{2}}{2}(1 + i) \\):  \n\\[\nz'' = (\\sqrt{2} - 3\\sqrt{2}i) \\cdot \x0crac{\\sqrt{2}}{2}(1 + i)\n\\]  \n\\[\n= \x0crac{1}{2}(2 - 6i + 2i + 6) = 1 + i - 3i + 3 = 4 - 2i\n\\]\n\n3. **Translate back:**  \n\\[\nw = z'' + c = (4 - 2i) + (2 - 3i) = 6 - 5i\n\\]\n\n**Final answer:**  \n\\[\n\x08oxed{6 - 5i}\n\\]\n

# ## Get started

# The input above is the task for you, please refer to the above content and give me the output.

# ### input
# To determine the smallest possible value of \\( c \\) for the function \\( y = a \\sin (bx + c) + d \\), we need to analyze the given graph and extract information about the parameters \\( a \\), \\( b \\), \\( c \\), and \\( d \\).\n\nFirst, observe the amplitude \\( a \\). The amplitude is the maximum deviation from the midline of the sine wave. From the graph, the maximum value is 3 and the minimum value is -1. The midline is the average of these values:\n\\[\n\\text{Midline} = \\frac{3 + (-1)}{2} = 1\n\\]\nThe amplitude \\( a \\) is the distance from the midline to the maximum value:\n\\[\na = 3 - 1 = 2\n\\]\n\nNext, determine the period of the sine wave. The period \\( T \\) is the length of one complete cycle of the sine wave. From the graph, the period appears to be \\( \\frac{2\\pi}{3} \\). The period of the sine function \\( y = a \\sin (bx + c) + d \\) is given by:\n\\[\nT = \\frac{2\\pi}{b}\n\\]\nSetting this equal to the observed period:\n\\[\n\\frac{2\\pi}{b} = \\frac{2\\pi}{3}\n\\]\nSolving for \\( b \\):\n\\[\nb = 3\n\\]\n\nNow, determine the vertical shift \\( d \\). The vertical shift is the value of the midline:\n\\[\nd = 1\n\\]\n\nFinally, determine the phase shift \\( c \\). The phase shift is the horizontal shift of the sine wave. The standard sine function \\( y = \\sin x \\) starts at \\( x = 0 \\). For the function \\( y = 2 \\sin (3x + c) + 1 \\), the phase shift is given by:\n\\[\n\\text{Phase shift} = -\\frac{c}{b} = -\\frac{c}{3}\n\\]\nFrom the graph, the sine wave starts at \\( x = -\\frac{\\pi}{6} \\). Therefore, the phase shift is:\n\\[\n-\\frac{c}{3} = -\\frac{\\pi}{6}\n\\]\nSolving for \\( c \\):\n\\[\nc = \\frac{\\pi}{2}\n\\]\n\nThus, the smallest possible value of \\( c \\) is:\n\\[\n\\boxed{\\frac{\\pi}{2}}\n\\]

# """

In [17]:
# CoT
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)

In [19]:
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=4096,
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

To determine the smallest possible value of \( c \) for the function \( y = a \sin (bx + c) + d \), we need to analyze the given graph and extract information about the parameters \( a \), \( b \), \( c \), and \( d \).

First, observe the amplitude \( a \). The amplitude is the maximum deviation from the midline of the sine wave. From the graph, the maximum value is 3 and the minimum value is -1. The midline is the average of these values:
\[
\text{Midline} = \frac{3 + (-1)}{2} = 1
\]
The amplitude \( a \) is the distance from the midline to the maximum value:
\[
a = 3 - 1 = 2
\]

Next, determine the period of the sine wave. The period \( T \) is the length of one complete cycle of the sine wave. From the graph, the period appears to be \( \frac{2\pi}{3} \). The period of the sine function \( y = a \sin (bx + c) + d \) is given by:
\[
T = \frac{2\pi}{b}
\]
Setting this equal to the observed period:
\[
\frac{2\pi}{b} = \frac{2\pi}{3}
\]
Solving for \( b \):
\[
b = 3
\]

Now, determine 

In [20]:
print(len(response))

1645


In [25]:
system = """To determine the smallest possible value of \( c \) for the function \( y = a \sin (bx + c) + d \), we analyze the graph to extract \( a \), \( b \), \( c \), and \( d \).

First, observe the amplitude \( a \). The maximum value is 3 and the minimum is -1, so the midline is:
\[
\text{Midline} = \frac{3 + (-1)}{2} = 1
\]
\[
a = 3 - 1 = 2
\]

Next, determine the period. From the graph, the period is \( \frac{2\pi}{3} \). Since
\[
\frac{2\pi}{b} = \frac{2\pi}{3} \Rightarrow b = 3
\]

Now, determine the vertical shift:
\[
d = 1
\]

Finally, determine the phase shift. The function is \( y = 2 \sin (3x + c) + 1 \), so the phase shift is:
\[
-\frac{c}{3}
\]
From the graph, the sine wave starts at \( x = -\frac{\pi}{6} \), so:
\[
-\frac{c}{3} = -\frac{\pi}{6} \Rightarrow c = \frac{\pi}{2}
\]

\[
\boxed{\frac{\pi}{2}}
\]"""

print(repr(system))

'To determine the smallest possible value of \\( c \\) for the function \\( y = a \\sin (bx + c) + d \\), we analyze the graph to extract \\( a \\), \\( b \\), \\( c \\), and \\( d \\).\n\nFirst, observe the amplitude \\( a \\). The maximum value is 3 and the minimum is -1, so the midline is:\n\\[\n\text{Midline} = \x0crac{3 + (-1)}{2} = 1\n\\]\n\\[\na = 3 - 1 = 2\n\\]\n\nNext, determine the period. From the graph, the period is \\( \x0crac{2\\pi}{3} \\). Since\n\\[\n\x0crac{2\\pi}{b} = \x0crac{2\\pi}{3} \\Rightarrow b = 3\n\\]\n\nNow, determine the vertical shift:\n\\[\nd = 1\n\\]\n\nFinally, determine the phase shift. The function is \\( y = 2 \\sin (3x + c) + 1 \\), so the phase shift is:\n\\[\n-\x0crac{c}{3}\n\\]\nFrom the graph, the sine wave starts at \\( x = -\x0crac{\\pi}{6} \\), so:\n\\[\n-\x0crac{c}{3} = -\x0crac{\\pi}{6} \\Rightarrow c = \x0crac{\\pi}{2}\n\\]\n\n\\[\n\x08oxed{\x0crac{\\pi}{2}}\n\\]'


In [ ]:
print(len(system))

811


: 